# TBL Model Development: Random Forest Regression

This is a sanity check, running the random forest regression with the same methods but seeing if the ten folds perform similarly as the original ten folds. 

In [1]:
# imports
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from joblib import Parallel, delayed


In [2]:
# Processing Data Function (switching to using radius)

def read_and_process_data(directory_path):
    data_frames = []
    columns_to_extract = ['radius_X', 'radius_Y', 'radius_Z', 'radius_Ox', 'radius_Oy', 'radius_Oz', 'Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz']

    # Assuming each cycle has exactly 356 data points
    total_data_points = 356

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(directory_path, file_name)
            participant = int(os.path.basename(file_path).split('_')[0])
            cycle_id = os.path.basename(file_path).split('_')[1].split('.')[0]  # Extract cycle_id
            intensity = cycle_id[:4]  # Extract the first four characters of cycle_id as intensity

            # Read data from CSV and select only the desired columns
            df = pd.read_csv(file_path, usecols=columns_to_extract)

            # Add participant ID, cycle_id, and participant_cycle_id as features
            df['Participant'] = participant
            df['Cycle_ID'] = cycle_id
            df['Participant_Cycle_ID'] = f"{participant}_{cycle_id}"
            df['Intensity'] = intensity

            # Since Data will get shuffled (idk why but shuffling makes the model so much better.. ??)
            # So thus, need to store original index values 
            df['Original_Index'] = df.index

            # # Add normalized_cycle_position
            # df['Normalized_Cycle_Position'] = df.index / (total_data_points - 1)

            # if (intensity == "HIIT"):
            #     df['Intensity'] = 90
            # else:
            #     df['Intensity'] = 50
            
            # df['Intensity'] = intensity  # this is either "HIIT" or "MICT"

            data_frames.append(df)

    # Concatenate all data frames
    processed_data = pd.concat(data_frames, ignore_index=True)

    # Merge with participant weights
    weights_df = pd.read_csv("Participant Weights.csv")
    weights_df['Weight'] = weights_df['Weight'].astype(float)
    weights_df['Wingspan'] = weights_df['Wingspan'].astype(float)
    processed_data = pd.merge(processed_data, weights_df, left_on='Participant', right_on='Participant')

    return processed_data


In [3]:
#  # Set up paths
# data_directory = "Processed Data for ML"

# # Read and process data
# data = read_and_process_data(data_directory)

# # Shuffle the data based on 'Participant_Cycle_ID'
# data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# # Get unique participants
# participants = data['Participant'].unique()

# # Randomly select 16 participants for the train set and 4 participants for the test set
# train_participants = np.random.choice(participants, size=16, replace=False)
# test_participants = np.setdiff1d(participants, train_participants)

# # Split the data into train and test based on the selected participants
# train_data = data[data['Participant'].isin(train_participants)]
# test_data = data[data['Participant'].isin(test_participants)]

# # Specify the output columns
# output_columns = ['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz']

# # Create X (input) and y (output) for train/validation/test
# X_train = train_data.drop(output_columns, axis=1)  # Dropping the output columns to create input
# y_train = train_data[output_columns]  # Creating output, each column will be a separate y

# X_test = test_data.drop(output_columns, axis=1)  # Dropping the output columns to create input
# y_test = test_data[output_columns]  # Creating output, each column will be a separate y

In [4]:
# Set up paths
data_directory = "Processed Data for ML"
output_folder = "Sanity Train and Test Data 10 Fold"

# Read and process data
data = read_and_process_data(data_directory)

# Shuffle the data
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Get unique participants
participants = data['Participant'].unique()

# Initialize KFold cross-validator with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=45)

# Create lists to store train and test sets for all folds
all_X_train_sets = []
all_y_train_sets = []
all_X_test_sets = []
all_y_test_sets = []

fold_number = 1

# Iterate over each fold
for train_index, test_index in kf.split(participants):
    train_participants = participants[train_index]
    test_participants = participants[test_index]
    
    # Split the data into train and test based on the selected participants
    train_data = data[data['Participant'].isin(train_participants)]
    test_data = data[data['Participant'].isin(test_participants)]
    
    # Specify the output columns
    output_columns = ['Fx', 'Fy', 'Fz', 'Tx', 'Ty', 'Tz']
    
    # Create X (input) and y (output) for train/validation/test
    X_train = train_data.drop(output_columns, axis=1)  # Dropping the output columns to create input
    y_train = train_data[output_columns]  # Creating output, each column will be a separate y
    
    X_test = test_data.drop(output_columns, axis=1)  # Dropping the output columns to create input
    y_test = test_data[output_columns]  # Creating output, each column will be a separate y
    
    # Store train and test data
    fold_folder = os.path.join(output_folder, f"Fold_{fold_number}")
    os.makedirs(fold_folder, exist_ok=True)
    
    # Save train and test data to CSV
    train_data.to_csv(os.path.join(fold_folder, "train_data.csv"), index=False)
    test_data.to_csv(os.path.join(fold_folder, "test_data.csv"), index=False)
  awqs 
    # Save participant numbers to text files
    np.savetxt(os.path.join(fold_folder, "train_participants.txt"), train_participants, fmt='%d')
    np.savetxt(os.path.join(fold_folder, "test_participants.txt"), test_participants, fmt='%d')
    
    # Append train and test sets to lists
    all_X_train_sets.append(X_train)
    all_y_train_sets.append(y_train)
    all_X_test_sets.append(X_test)
    all_y_test_sets.append(y_test)
    
    fold_number += 1

In [5]:
# # train models

# # Define numerical and categorical features
# numeric_features = ['radius_X', 'radius_Y', 'radius_Z', 'radius_Ox', 'radius_Oy', 'radius_Oz', 'Normalized_Cycle_Position', 'Weight', 'Wingspan']
# categorical_features = ['Intensity']

# # Create transformers for numerical and categorical features
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Create a preprocessor that applies transformers to specific columns 
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ]
# )

# # Create a pipeline with the preprocessor and the regressor
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', RandomForestRegressor())
# ])

# # Define the parameter grid for GridSearchCV
# param_grid = {
#     'regressor__n_estimators': [400, 450],  # Number of trees in the forest
# }

# # Create a GridSearchCV object
# # grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# # Fit the GridSearchCV object to your training data
# # grid_search.fit(X_train, y_train['Fx'])

# # Create a GridSearchCV object
# grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# # Fit the GridSearchCV object to your training data in parallel
# with Parallel(n_jobs=-1):
#     grid_search.fit(X_train, y_train['Fx'])

# # Access the best parameters and best model
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# # report model metrics

# print("Best Parameters:", best_params)

# # Get the mean cross-validated "accuracy" score of the best model
# best_score = grid_search.best_score_

# print("Mean Accuracy of Best Model (in Validation):", best_score)

In [6]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from joblib import Parallel, delayed

# # Define numerical and categorical features
# # numeric_features = ['radius_X', 'radius_Y', 'radius_Z', 'radius_Ox', 'radius_Oy', 'radius_Oz', 'Normalized_Cycle_Position', 'Weight', 'Wingspan']
# # Trying model without Normalized Cycle Position
# numeric_features = ['radius_X', 'radius_Y', 'radius_Z', 'radius_Ox', 'radius_Oy', 'radius_Oz', 'Weight', 'Wingspan']
# categorical_features = ['Intensity'] # i can try making numerical --> it worsened performance

# # Create transformers for numerical and categorical features
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Create a preprocessor that applies transformers to specific columns 
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ]
# )

# # Define the parameter grid for GridSearchCV (training on just 400 (because that's the rough optimal from Fx training))
# param_grid = {
#     'regressor__n_estimators': [400],  # Number of trees in the forest
# }

# # Initialize dictionary to store best models
# best_models = {}

# # Train a model for each output column
# for output_col in output_columns:
#     # Create a pipeline with the preprocessor and the regressor
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('regressor', RandomForestRegressor())
#     ])

#     # Create a GridSearchCV object
#     grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

#     # Fit the GridSearchCV object to your training data in parallel
#     with Parallel(n_jobs=-1):
#         grid_search.fit(X_train, y_train[output_col])

#     # Access the best model
#     best_models[output_col] = grid_search.best_estimator_

#     # report model metrics
#     print("Output Column:", output_col)
#     print("Best Parameters:", grid_search.best_params_)
#     print("Neg Mean Squared Error of Best Model (in Validation):", grid_search.best_score_)



In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import gc  # Import the garbage collector module

# Define numerical features
numeric_features = ['radius_X', 'radius_Y', 'radius_Z', 'radius_Ox', 'radius_Oy', 'radius_Oz', 'Weight', 'Wingspan']
categorical_features = ['Intensity']

# Create transformers for numerical and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create a preprocessor that applies transformers to specific columns 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Initialize a dictionary to store best models for each fold
best_models = {}

# Define RandomForestRegressor with n_estimators=400
regressor = RandomForestRegressor(n_estimators=400, n_jobs=-1)

# Iterate over each fold
for fold_number in range(1, 11):
    X_train = all_X_train_sets[fold_number - 1]
    y_train = all_y_train_sets[fold_number - 1]
    
    # Clear memory before starting a new fold
    gc.collect()
    
    # Create a pipeline with the preprocessor and the regressor
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', regressor)
    ])
    
    # Fit the pipeline to the training data
    pipeline.fit(X_train, y_train)
    
    # Store the best model for this output column in the corresponding fold's dictionary
    print(f"\nFold {fold_number} completed.")
    
    best_models[str(fold_number)] = pipeline
    
    # Clear memory after completing a fold
    del X_train, y_train, pipeline
    gc.collect()


Fold 1 completed.

Fold 2 completed.

Fold 3 completed.

Fold 4 completed.

Fold 5 completed.

Fold 6 completed.

Fold 7 completed.

Fold 8 completed.

Fold 9 completed.

Fold 10 completed.


In [8]:
def calculate_mdpe_by_participant_cycle_ids(y_pred, y_test, X_test):
    """
    Calculate Median Percentage Error (MDPE) for each unique participant_cycle_id for a specific output column.
    
    Args:
    - y_pred (numpy array): Predicted values.
    - y_test (numpy array): True values.
    - X_test (DataFrame): DataFrame containing the test data including 'Participant_Cycle_ID'.
    - output_column (str): Name of the output column.
    
    Returns:
    - mdpe_scores (dict): Dictionary containing MDPE scores for each unique participant_cycle_id.
    """
    mdpe_scores = []
    
    # Get unique participant_cycle_ids
    unique_participant_cycle_ids = X_test['Participant_Cycle_ID'].unique()
    
    # Calculate MDPE for each unique participant_cycle_id
    for unique_id in unique_participant_cycle_ids:
        mask = X_test['Participant_Cycle_ID'] == unique_id
        y_pred_id = y_pred[mask]
        y_test_id = y_test[mask]
        
        # Exclude NaN values
        mask_valid = ~np.isnan(y_test_id)
        y_pred_id = y_pred_id[mask_valid]
        y_test_id = y_test_id[mask_valid]
        
        mdpe = np.median((y_pred_id - y_test_id) / y_test_id * 100)
        mdpe_scores.append(mdpe)
        
    return mdpe_scores

In [9]:
# # getting MDPE based on test data set

# # Initialize y_pred as a dictionary with keys for each output column
# y_pred = {output_col: [] for output_col in output_columns}

# mdpe_scores_outputs = {}
# plot_data = []

# for output_col in output_columns:
#     y_pred[output_col] = best_models[output_col].predict(X_test)
#     mdpe_scores = calculate_mdpe_by_participant_cycle_ids(y_pred, y_test, X_test, output_col)
#     mdpe_scores_outputs[output_col] = mdpe_scores
#     average = np.mean(mdpe_scores)
#     std = np.std(mdpe_scores)
#     print("Output Column:", output_col)
#     print("Average of MdPEs:", average)
#     print("Standard Deviation of MdPEs:", std)
#     # Add data for plotting
#     for score in mdpe_scores:
#         plot_data.append({'Output': output_col, 'MDPE': score})



# # Create a box-and-whisker plot with stripplot
# plt.figure(figsize=(10, 6))
# box_plot = sns.boxplot(x='Output', y='MDPE', data=pd.DataFrame(plot_data))
# strip_plot = sns.stripplot(x='Output', y='MDPE', data=pd.DataFrame(plot_data), jitter=True, color=".25", marker='o',
#                             size=2)  # Add individual data points with smaller dots
# plt.title('Box-and-Whisker Plot of MDPE Scores for Each Output Column')

# # Customize legend for the strip plot
# legend_labels = {'Individual Data Points': 'o'}
# handles, _ = strip_plot.get_legend_handles_labels()
# box_plot.legend(handles, legend_labels.values(), title='Legend', loc='upper right')

# plt.show()

In [10]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize y_pred as a dictionary with keys for each fold number
y_pred = {fold_number: [] for fold_number in range(1, 11)}

# Create a directory to store the y_pred for each fold and each output
output_directory = "Sanity Random Forest Outputs"
os.makedirs(output_directory, exist_ok=True)

mdpe_scores_outputs = {}

# Accumulate MDPE scores across all folds for each output variable
combined_mdpe_scores = {output_col: [] for output_col in output_columns}

# Iterate over each fold
for fold_number in range(1, 11):
    X_test = all_X_test_sets[fold_number - 1]
    y_test = all_y_test_sets[fold_number - 1]
    
    print(f"\nFold {fold_number}:")
    
    # Predict y_pred for all output columns in this fold
    y_pred[fold_number] = best_models[str(fold_number)].predict(X_test)

    # Save selected columns from X_test, y_test, and y_pred for this fold as CSV
    fold_output_directory = os.path.join(output_directory, f"Fold_{fold_number}")
    os.makedirs(fold_output_directory, exist_ok=True)

    # Convert the NumPy array to a pandas DataFrame
    y_pred_df = pd.DataFrame(y_pred[fold_number], columns=output_columns)
    
    # Save train and test data to CSV
    X_test.to_csv(os.path.join(fold_output_directory, "X_test.csv"), index=False)
    y_test.to_csv(os.path.join(fold_output_directory, "y_test.csv"), index=False)
    y_pred_df.to_csv(os.path.join(fold_output_directory, "y_pred.csv"), index=False)

    # Calculate MDPE scores for this fold and each output column
    mdpe_scores_list = []
    for i, output_col in enumerate(output_columns):
        y_pred_fold_output = y_pred[fold_number][:, i]
        
        mdpe_scores = calculate_mdpe_by_participant_cycle_ids(y_pred_fold_output, y_test[output_col], X_test)
        mdpe_scores_outputs[(fold_number, output_col)] = mdpe_scores
        mdpe_scores_list.append(pd.DataFrame({'Output': [output_col]*len(mdpe_scores), 'MDPE': mdpe_scores}))
        
        # Append MDPE scores to combined_mdpe_scores
        combined_mdpe_scores[output_col].extend(mdpe_scores)
        
        average = np.mean(mdpe_scores)
        std = np.std(mdpe_scores)
        print(f"Output Column: {output_col}")
        print("Average of MdPEs:", average)
        print("Standard Deviation of MdPEs:", std)
        
        # Save average and standard deviation of MdPEs to the output directory
        with open(os.path.join(output_directory, f"Fold_{fold_number}_mdpe_scores.txt"), 'a') as f:
            f.write(f"Output Column: {output_col}\n")
            f.write(f"Average of MdPEs: {average}\n")
            f.write(f"Standard Deviation of MdPEs: {std}\n\n")
    
    # Combine MDPE scores for all output columns in this fold
    mdpe_df = pd.concat(mdpe_scores_list)
    
    # Plot MDPE scores for this fold and all output columns
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='Output', y='MDPE', data=mdpe_df, palette='husl')  # Using 'husl' palette for more colorful plots
    plt.title(f'Box-and-Whisker Plot of MDPE Scores for Fold {fold_number}')
    plt.ylabel('MDPE')
    plt.xlabel('Output Column')
    plt.xticks(rotation=45)
    plt.savefig(os.path.join(fold_output_directory, f"MDPE_plot.png"))
    plt.close()

# Calculate the mean and standard deviation of MDPE scores for each output variable across all folds
average_mdpe_scores = {}
std_mdpe_scores = {}

for output_col in output_columns:
    average_mdpe_scores[output_col] = np.mean(combined_mdpe_scores[output_col])
    std_mdpe_scores[output_col] = np.std(combined_mdpe_scores[output_col])

# Write mean and standard deviation of MDPEs to a txt file
with open(os.path.join(output_directory, "average_mdpe_scores.txt"), 'w') as f:
    for output_col in output_columns:
        f.write(f"Output Column: {output_col}\n")
        f.write(f"Average of MdPEs across 10 folds: {average_mdpe_scores[output_col]}\n")
        f.write(f"Standard Deviation of MdPEs across 10 folds: {std_mdpe_scores[output_col]}\n\n")

# Combine MDPE scores for all output columns across all folds
combined_mdpe_df = pd.concat([pd.DataFrame({'Output': [output_col]*len(combined_mdpe_scores[output_col]), 'MDPE': combined_mdpe_scores[output_col]}) for output_col in output_columns], ignore_index=True)

# Plot combined MDPE scores for all output columns
plt.figure(figsize=(12, 8))
sns.boxplot(x='Output', y='MDPE', data=combined_mdpe_df, palette='husl')  # Using 'husl' palette for more colorful plots
plt.title('Box-and-Whisker Plot of Combined MDPE Scores for All Folds')
plt.ylabel('MDPE')
plt.xlabel('Output Column')
plt.xticks(rotation=45)
plt.savefig(os.path.join(output_directory, "combined_MDPE_plot.png"))
plt.close()


Fold 1:


Output Column: Fx
Average of MdPEs: -0.29953661889290845
Standard Deviation of MdPEs: 0.8670804318920072
Output Column: Fy
Average of MdPEs: -0.3194180983720691
Standard Deviation of MdPEs: 0.6598036797663287
Output Column: Fz
Average of MdPEs: -1.174144183134095
Standard Deviation of MdPEs: 2.020416837797887
Output Column: Tx
Average of MdPEs: -0.331117668732371
Standard Deviation of MdPEs: 0.8938324478703472
Output Column: Ty
Average of MdPEs: -0.09005792323648332
Standard Deviation of MdPEs: 0.8991309948830266
Output Column: Tz
Average of MdPEs: -0.04144161096046334
Standard Deviation of MdPEs: 1.4032463239258033

Fold 2:
Output Column: Fx
Average of MdPEs: -0.09499124598530927
Standard Deviation of MdPEs: 0.7845883895461461
Output Column: Fy
Average of MdPEs: -0.24103283697641711
Standard Deviation of MdPEs: 0.5139802889315729
Output Column: Fz
Average of MdPEs: -1.0030907535561895
Standard Deviation of MdPEs: 1.3547039727013443
Output Column: Tx
Average of MdPEs: -0.25281039974504